# Coding Exercise

You must correctly implement the function described in the prompt below.

Feel free to test out pieces of code to help you write the solution.

Please thoroughly test that the final code implements the function correctly.

## Prompt

**Function signature:** `arrangements(N: int, visible: List[int], hidden: int) -> int`

    Note to plugin users: there are html elements in this statement that will not appear correct in plugins.  Please use the statement to view them.

    Russian Nesting Dolls are a set of dolls which nest inside each other.  The dolls are constructed in such a way that each doll will fit comfortably inside any larger doll, and no two dolls are the same size.  There are a finite number of ways to nest dolls inside one another so that a specific subset of the dolls is visible.  For example, with 3 dolls, there is only one way that they could be nested such that only the largest doll is visible, but there are 2 ways to nest the dolls if the largest and medium doll are visible (depending on which of the two larger dolls the smallest doll is inside).  In addition, if you are just given how many dolls are visible, there may be even more ways that the dolls could be nested.  For example, if you know there are 2 dolls visible in a 3-doll set, there are 3 ways they could be nested (see example 0).

    Your clever friend has used an entire set of N nesting dolls in an arrangement on a table.  He assures you that all the nesting dolls are on the table, but you can only see a subset of the dolls.  In addition, he has placed a piece of cardboard in front of some of the dolls so that you cannot see which ones they are, and he tells you how many are behind the board.  He asks you how many possible ways he could have arranged the dolls such that the current configuration is on the table.

    Each doll has a number painted on it from 1 to N, which identifies the size of the doll.  You can fit a doll labeled i inside a doll labeled j as long as i < j.  Your program will be given an int N, identifying how many dolls there are in the set, a List[int] visible, identifying which dolls you can see on the table, and an int hidden, identifying how many more dolls would be visible if the board was removed.  Two arrangements are different if any doll is nested immediately inside a different doll, or is nested in one arrangement but not in the other.

    Notes
    -The constraints are constructed so the return value will be <= 263 - 1.
 
    Constraints
    -N will be between 2 and 25, inclusive.
    -hidden will be between 0 and N, inclusive.
    -visible will have between 0 and N-hidden elements, inclusive.
    -Each element of visible will be between 1 and N, inclusive.
    -There will be no repeated elements in visible.
    -There will be at least one configuration which is possible given the arguments.
 
    Examples
    0)
        3
    {}
    2

    Returns: 3
    There are three dolls, two of which are behind the board.  Since no dolls are showing, the third doll must be nested inside one of the other two.  There are three possibilities:
    1 and 3 are behind the board, 2 is nested inside 3
    2 and 3 are behind the board, 1 is nested inside 2
    2 and 3 are behind the board, 1 is nested inside 3

    1)
        3
    {2}
    1

    Returns: 2
    In this case, doll 2 is showing.  Since there is only one doll behind the board, it must be doll 3.  The only unknown is doll 1.  It could either be nested in doll 3 or doll 2.

    2)
        9
    {9}
    1

    Returns: 255
    Dolls 1-8 could either be behind the board or inside doll 9.  However, the case where 1-8 are all inside of 9 is not possible, because then there wouldn't be a doll behind the board.  So the answer is 28 - 1.

    3)
        20
    {13,5,2,18}
    3

    Returns: 7163268480

    

The constraints are constructed so the output is smaller than 263-1, therefore I use brute force. 

The brute force algorithm is to simply generate all possible configurations.
We will generate the configurations recursively by distributing the the largest dolls to the smallest.
The state of the table will be represented by a 2d list[space][dolls], where the first index represent a space on the table, and the second index the dolls in the space.

In [3]:
from typing import List
import copy

def get_configs(N, table, available):
    if sum(available) == 0:
        return [table]
    
    table_list = []
    
    #get last doll and set it to not available
    last = len(available) - 1 - available[::-1].index(True)
    available[last] = False
    
    #try to fit the doll at each space on the table
    for i in range(len(table)):
        if min(table[i]) > last: #condition to check if doll fit inside
            t = copy.deepcopy(table)
            t[i].append(last)
            table_list.append(get_configs(N, t, available))
            
    return table_list

def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = []*len(visible)+[]*hidden
    for i, v in enumerate(visible):
        table[i].append(v)
    
    #array to check if doll is not on the table
    available = [False if v in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    return len(configs), configs

arrangements(3, [], 2)

(0, [])

The result is clearly wrong (the first result should be 3).
The problem is that the table should be 2d lists

In [4]:
def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[]]*len(visible)+[[]]*hidden
    for i, v in enumerate(visible):
        table[i].append(v)
    
    available = [False if v in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    return len(configs), configs

arrangements(3, [], 2)

ValueError: min() arg is an empty sequence

The 2d list cannot start empty. Therefore I will start it with a super large doll of size 100. 

In [5]:
def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[100]]*len(visible)+[[100]]*hidden
    for i, v in enumerate(visible):
        table[i].append(v)
    
    available = [False if v in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    return len(configs), configs

s, conf = arrangements(3, [], 2)
print(s)
print(conf)

2
[[[[[[100, 2, 1, 0], [100, 2, 1, 0]]], [[[100, 2, 1, 0], [100, 2, 1, 0]]]], [[[100, 2, 1], [100, 2, 1]]]], [[[100, 2], [100, 2]]]]


Fixing get_configs to return a list of tables, so our recursion results becomes consitent

In [6]:
def get_configs(N, table, available):
    if sum(available) == 0:
        return [table]
    
    table_list = []
    
    last = len(available) - 1 - available[::-1].index(True)
    available[last] = False
    
    for i in range(len(table)):
        if min(table[i]) > last:
            t = copy.deepcopy(table)
            a = copy.deepcopy(available)
            t[i].append(last)
            table_list += get_configs(N, t, a)
            
    return table_list

s, conf = arrangements(3, [], 2)
print(s)
for c in conf:
    print(c)

8
[[100, 2, 1, 0], [100, 2, 1, 0]]
[[100, 2, 1, 0], [100, 2, 1, 0]]
[[100, 2, 1, 0], [100, 2, 1, 0]]
[[100, 2, 1, 0], [100, 2, 1, 0]]
[[100, 2, 1, 0], [100, 2, 1, 0]]
[[100, 2, 1, 0], [100, 2, 1, 0]]
[[100, 2, 1, 0], [100, 2, 1, 0]]
[[100, 2, 1, 0], [100, 2, 1, 0]]


The results are clearly wrong, and I honestly have no ideia why.
Looks like I am not coping the table correctly, but I do not know what is wrong.

In [7]:
def get_configs(N, table, available):
    
    if sum(available) == 0:
        return [copy.deepcopy(table)]
    
    table_list = []
    
    last = len(available) - 1 - available[::-1].index(True)
    available = copy.deepcopy(available)
    available[last] = False
    
    for i in range(len(table)):
        if min(table[i]) > last and i != 1:
            print(table)
            t = copy.deepcopy(table)
            a = copy.deepcopy(available)
            t[i].append(last)
            table_list += get_configs(N, t, a)
            break
            
    return table_list

s, conf = arrangements(3, [1], 2)
print(s)
print(conf)

[[100, 1], [100], [100]]
[[100, 1], [100, 2], [100, 2]]
1
[[[100, 1, 0], [100, 2], [100, 2]]]


In [8]:
sum([False, False, False])

0

In [9]:
a = [[100]]*10
a

[[100], [100], [100], [100], [100], [100], [100], [100], [100], [100]]

In [10]:
a[0].append(54)
a

[[100, 54],
 [100, 54],
 [100, 54],
 [100, 54],
 [100, 54],
 [100, 54],
 [100, 54],
 [100, 54],
 [100, 54],
 [100, 54]]

After spending a long time thinking, I realized that my initialization of the table is incorrect, as shown in the two cells above.
I will fix the table initialization in function arrangements (with two for loops) so they are initialized properly.

In [11]:
def get_configs(N, table, available):
    
    if sum(available) == 0:
        return [copy.deepcopy(table)]
    
    table_list = []
    
    last = len(available) - 1 - available[::-1].index(True)
    available = copy.deepcopy(available)
    available[last] = False
    
    for i in range(len(table)):
        if min(table[i]) > last:
            t = copy.deepcopy(table)
            a = copy.deepcopy(available)
            t[i].append(last)
            table_list += get_configs(N, t, a)
            
    return table_list

def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[100] for i in range(len(visible))]+[[100] for i in range(hidden)]
    for i, v in enumerate(visible):
        table[i].append(v)
    
    available = [False if v in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    return len(configs), configs

s, conf = arrangements(3, [], 2)
print(s)
print(conf)

8
[[[100, 2, 1, 0], [100]], [[100, 2, 1], [100, 0]], [[100, 2, 0], [100, 1]], [[100, 2], [100, 1, 0]], [[100, 1, 0], [100, 2]], [[100, 1], [100, 2, 0]], [[100, 0], [100, 2, 1]], [[100], [100, 2, 1, 0]]]


I am returning all the possible results, however we need unique results only. I will try to filter the unique results by sorting the lists, casting them to strings and converting to a set.

In [12]:
def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[100] for i in range(len(visible))]+[[100] for i in range(hidden)]
    for i, v in enumerate(visible):
        table[i].append(v)
    
    available = [False if v in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    configs.sort()
    configs = set([str(c) for c in configs])
    
    return len(configs), configs

s, conf = arrangements(3, [], 2)
print(s)
print(conf)

8
{'[[100, 2, 1], [100, 0]]', '[[100, 1, 0], [100, 2]]', '[[100, 2], [100, 1, 0]]', '[[100, 2, 0], [100, 1]]', '[[100, 0], [100, 2, 1]]', '[[100], [100, 2, 1, 0]]', '[[100, 2, 1, 0], [100]]', '[[100, 1], [100, 2, 0]]'}


It clearly did not work. I need to sort every possible configuration inside the list, sort the list, and them I can cast every configuration to string and convert them to a set.

In [13]:
def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[100] for i in range(len(visible))]+[[100] for i in range(hidden)]
    for i, v in enumerate(visible):
        table[i].append(v)
    
    available = [False if v in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    configs = [sorted(c) for c in configs]
    configs.sort()
    configs = set([str(c) for c in configs])
    
    return len(configs), configs

s, conf = arrangements(3, [], 2)
print(s)
print(conf)

4
{'[[100], [100, 2, 1, 0]]', '[[100, 1, 0], [100, 2]]', '[[100, 1], [100, 2, 0]]', '[[100, 0], [100, 2, 1]]'}


Much better! Now I need to remove entries that have only one 100. A doll of size 100 was artificially inserted to fill the initialization of the algorithm. 

In [14]:
def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[100] for i in range(len(visible))]+[[100] for i in range(hidden)]
    for i, v in enumerate(visible):
        table[i].append(v)
    
    available = [False if v in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    configs = [sorted(c) for c in configs]
    configs = [c for c in configs if len(c[0]) > 1]
    configs.sort()
    configs = set([str(c) for c in configs])
    
    return len(configs), configs

s, conf = arrangements(3, [], 2)
print(s)
print(conf)

3
{'[[100, 1, 0], [100, 2]]', '[[100, 1], [100, 2, 0]]', '[[100, 0], [100, 2, 1]]'}


looks like it is finally working 

In [15]:
s, conf = arrangements(3, [2], 1)
print(s)
print(conf)

3
{'[[100, 1, 0], [100, 2]]', '[[100, 1], [100, 2, 0]]', '[[100, 0], [100, 2, 1]]'}


I am counting dolls in the wrong way. The prompt count dolls starting from 1, but my algorithm counts from 0.
The fix is simple and I only need to change the results related to the variable visible.

In [19]:
def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[100] for i in range(len(visible))]+[[100] for i in range(hidden)]
    for i, v in enumerate(visible):
        table[i].append(v-1)
    
    available = [False if v+1 in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    configs = [sorted(c) for c in configs]
    configs = [c for c in configs if len(c[0]) > 1]
    configs.sort()
    configs = set([str(c) for c in configs])
    
    return len(configs), configs

s, conf = arrangements(3, [2], 1)
print(s)
print(conf)

2
{'[[100, 1, 0], [100, 2]]', '[[100, 1], [100, 2, 0]]'}


In [20]:
s, conf = arrangements(9, [9], 1)
print(s)
print(conf)

255
{'[[100, 6, 0], [100, 8, 7, 5, 4, 3, 2, 1]]', '[[100, 7, 6, 5, 3, 0], [100, 8, 4, 2, 1]]', '[[100, 7, 6, 5, 2], [100, 8, 4, 3, 1, 0]]', '[[100, 6, 5, 3, 2, 1, 0], [100, 8, 7, 4]]', '[[100, 5, 3, 1], [100, 8, 7, 6, 4, 2, 0]]', '[[100, 7, 5, 4, 1], [100, 8, 6, 3, 2, 0]]', '[[100, 6, 2, 1, 0], [100, 8, 7, 5, 4, 3]]', '[[100, 7, 6, 4, 2], [100, 8, 5, 3, 1, 0]]', '[[100, 7, 2, 1], [100, 8, 6, 5, 4, 3, 0]]', '[[100, 6, 4, 3, 2, 1, 0], [100, 8, 7, 5]]', '[[100, 6, 4, 3, 2], [100, 8, 7, 5, 1, 0]]', '[[100, 6, 3], [100, 8, 7, 5, 4, 2, 1, 0]]', '[[100, 7, 2, 1, 0], [100, 8, 6, 5, 4, 3]]', '[[100, 3, 2, 0], [100, 8, 7, 6, 5, 4, 1]]', '[[100, 7, 5, 3, 1, 0], [100, 8, 6, 4, 2]]', '[[100, 7, 6, 5], [100, 8, 4, 3, 2, 1, 0]]', '[[100, 7, 5, 4], [100, 8, 6, 3, 2, 1, 0]]', '[[100, 6, 4, 3, 0], [100, 8, 7, 5, 2, 1]]', '[[100, 7, 4], [100, 8, 6, 5, 3, 2, 1, 0]]', '[[100, 6, 5, 4, 1, 0], [100, 8, 7, 3, 2]]', '[[100, 7, 5, 1], [100, 8, 6, 4, 3, 2, 0]]', '[[100, 5, 2, 0], [100, 8, 7, 6, 4, 3, 1]]', '[[10

Nice, Let me summarize the final implementation and tests. 

In [21]:
def get_configs(N, table, available):
    
    if sum(available) == 0:
        return [copy.deepcopy(table)]
    
    table_list = []
    
    last = len(available) - 1 - available[::-1].index(True)
    available = copy.deepcopy(available)
    available[last] = False
    
    for i in range(len(table)):
        if min(table[i]) > last:
            t = copy.deepcopy(table)
            a = copy.deepcopy(available)
            t[i].append(last)
            table_list += get_configs(N, t, a)
            
    return table_list

def arrangements(N: int, visible: List[int], hidden: int) -> int:
    #representation of the table
    table = [[100] for i in range(len(visible))]+[[100] for i in range(hidden)]
    for i, v in enumerate(visible):
        table[i].append(v-1)
    
    available = [False if v+1 in visible else True for v in range(N)]
        
    configs = get_configs(N, table, available)
    configs = [sorted(c) for c in configs]
    configs = [c for c in configs if len(c[0]) > 1]
    configs.sort()
    configs = set([str(c) for c in configs])
    
    return len(configs)

assert arrangements(3, {}, 2) == 3
assert arrangements(3, {2}, 1) == 2
assert arrangements(9, {9}, 1) == 255
print('done')

done


Accoding to the prompt, the result should be smaller than 263-1.
Therefore I do not expect that my algorithm will be able to solve the last example.

In [22]:
from datetime import datetime

t1 = datetime.now()
print(arrangements(20, {13,5,2,18}, 3))
t2 = datetime.now()
print(t2, t1)

KeyboardInterrupt: 